In [1]:
from cosim import Manager, Kio

# Setting up custom infrastructure

* We require an underlying infrastructure of compute nodes that can execute tasks and workflows

* create a new set of nodes with 3 offloading locations - `ue`, `edge` and `cloud`

In [2]:
infra = Manager.NewInfra(['ue', 'edge', 'cloud'])
infra

{'ue': {'name': 'ue',
  'nhost': '<notify-ip>',
  'nport': '<notify-port>',
  'dhost': '<data-ip>',
  'dport': '<data-port>',
  'https': False,
  'xy': (0.0, 0.0)},
 'edge': {'name': 'edge',
  'nhost': '<notify-ip>',
  'nport': '<notify-port>',
  'dhost': '<data-ip>',
  'dport': '<data-port>',
  'https': False,
  'xy': (0.0, 0.0)},
 'cloud': {'name': 'cloud',
  'nhost': '<notify-ip>',
  'nport': '<notify-port>',
  'dhost': '<data-ip>',
  'dport': '<data-port>',
  'https': False,
  'xy': (0.0, 0.0)}}

* set appropriate values to the `infra` object attributes

In [3]:
# for this example, use the same localhost but different ports

infra['ue'].update(dict(nhost="127.0.0.1", dhost="127.0.0.1", nport='9800', dport='9801'))
infra['edge'].update(dict(nhost="127.0.0.1", dhost="127.0.0.1", nport='9810', dport='9811',))
infra['cloud'].update(dict(nhost="127.0.0.1", dhost="127.0.0.1", nport='9820', dport='9821',))

infra

{'ue': {'name': 'ue',
  'nhost': '127.0.0.1',
  'nport': '9800',
  'dhost': '127.0.0.1',
  'dport': '9801',
  'https': False,
  'xy': (0.0, 0.0)},
 'edge': {'name': 'edge',
  'nhost': '127.0.0.1',
  'nport': '9810',
  'dhost': '127.0.0.1',
  'dport': '9811',
  'https': False,
  'xy': (0.0, 0.0)},
 'cloud': {'name': 'cloud',
  'nhost': '127.0.0.1',
  'nport': '9820',
  'dhost': '127.0.0.1',
  'dport': '9821',
  'https': False,
  'xy': (0.0, 0.0)}}

* save this to a json file

In [4]:
Kio.SaveJSON('__infra__.json', infra)

#### Notes:

* we need to start the `notify` and `data` scripts - these are webservers that are used to notify offloading decision to nodes and transfer data between them respectively.

* Note: we can use a single server for `notify` and `data` by specifying the same `nport` and `dport`. However using two spereate instance is recomended as the `notify` script is supposed to be single threaded. It should not interfere with `data` transfer which can use mutiple threads on dedicated ports.



* In this example, we use the same host
* create a folder named `__infra__`

In [5]:
import os
os.makedirs("__infra__", exist_ok=True)

* use following commands to start required scripts on devices:

In [6]:
for i,a in enumerate( list(infra.keys())):
    print(f"""
python -m cosim.work \\
--base=__infra__/{a} \\
--infra=__infra__.json \\
--script=.venv/bin/python \\
--mods=__mods__ \\
--host={infra[a]['nhost']} --port={infra[a]['nport']} --log=log.txt --threads=1 --maxupsize=10MB 

python -m cosim.work \\
--base=__infra__/{a} \\
--infra=__infra__.json \\
--script=.venv/bin/python \\
--mods=__mods__ \\
--host={infra[a]['dhost']} --port={infra[a]['dport']} --log=data.txt --threads=4 --maxupsize=500GB 
""")



python -m cosim.work \
--base=__infra__/ue \
--infra=__infra__.json \
--script=.venv/bin/python \
--mods=__mods__ \
--host=127.0.0.1 --port=9800 --log=log.txt --threads=1 --maxupsize=10MB 

python -m cosim.work \
--base=__infra__/ue \
--infra=__infra__.json \
--script=.venv/bin/python \
--mods=__mods__ \
--host=127.0.0.1 --port=9801 --log=data.txt --threads=4 --maxupsize=500GB 


python -m cosim.work \
--base=__infra__/edge \
--infra=__infra__.json \
--script=.venv/bin/python \
--mods=__mods__ \
--host=127.0.0.1 --port=9810 --log=log.txt --threads=1 --maxupsize=10MB 

python -m cosim.work \
--base=__infra__/edge \
--infra=__infra__.json \
--script=.venv/bin/python \
--mods=__mods__ \
--host=127.0.0.1 --port=9811 --log=data.txt --threads=4 --maxupsize=500GB 


python -m cosim.work \
--base=__infra__/cloud \
--infra=__infra__.json \
--script=.venv/bin/python \
--mods=__mods__ \
--host=127.0.0.1 --port=9820 --log=log.txt --threads=1 --maxupsize=10MB 

python -m cosim.work \
--base=__infr

#### Offloader (offloading Manager)

In [7]:

print("""

python -m cosim.work \\
--base=__infra__/O \\
--infra=__infra__.json \\
--script=.venv/bin/python \\
--mods=__mods__ \\
--host=127.0.0.1 --port=9696 --log=off.txt --threads=1 --maxupsize=10MB 

""")




python -m cosim.work \
--base=__infra__/O \
--infra=__infra__.json \
--script=.venv/bin/python \
--mods=__mods__ \
--host=127.0.0.1 --port=9696 --log=off.txt --threads=1 --maxupsize=10MB 




* Run the above commands in different terminals
* after this the nodes are ready for offloading